In [1]:
import pandas as pd
import seaborn as sea
import numpy as np
from matplotlib import pyplot as plt
from statsmodels.tsa.stattools import grangercausalitytests

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('Data/State_Consolidated_Mobility_Miles_New_Actionable_Oct20.csv')
df.fillna(0, inplace=True)

In [17]:
def generateCausations(df):
    try:
        tmp= grangercausalitytests(df[['desCartes','Incidence']], maxlag=1)
        return round(tmp[1][0]['ssr_ftest'][0],2),round(tmp[1][0]['ssr_ftest'][1],2)
    except:
        return -1,-1


def enterDict(state, lag, phase, correlation, pvalue):
    row = {}
    row['state'] = state
    row['lag'] = lag
    row['Phase'] = phase
    row['Causality_F'] = correlation
    row['Causality_P'] = pvalue
    return row


def normalize(series):
    return (series-series.min())/(series.max()-series.min())

In [18]:
states = df['state'].unique()
phdf = pd.read_csv('Data/phases_phase1.csv')
ndf = {}

sdf = pd.DataFrame()
k = 0
lags = [-10,-9,-8,-7,-6,-5,-4,-3,-2,-1]
#lags = [-2]
for state in states:
    print(state)
    for lag in lags:

        tdf = df[df['state']==state]
        tdf['desCartes'] = tdf['desCartes'].shift((lag*-1))
        tdf = tdf[~tdf['desCartes'].isin([np.nan, np.inf, -np.inf])]
        tdf = tdf[~tdf['Incidence'].isin([np.nan, np.inf, -np.inf])]
        row={}
        row['state'] = state
        row['Phase'] = 'CompleteSeries'
        print( tdf['desCartes'].values, tdf['Incidence'].values)
        row['correlation'] = tdf['desCartes'].corr(tdf['Incidence'], method='spearman')
        tmp= grangercausalitytests(tdf[['desCartes','Incidence']], maxlag=1)
        row['Causality_F'] = round(tmp[1][0]['ssr_ftest'][0],2)
        row['Causality_P'] = round(tmp[1][0]['ssr_ftest'][1],2)

        row['lag'] = lag
        ndf[k] =row
        k = k+1
        tdf.sort_values('startDayWeek', inplace=True)
        if(state  in phdf.columns):
            tphdf = phdf[['Phase Loc',state]]
            p1_start = tphdf[tphdf['Phase Loc']=='w1_tp1'][state].values[0]
            p1_end = tphdf[tphdf['Phase Loc']=='w1_tp2'][state].values[0]
            p2_start = tphdf[tphdf['Phase Loc']=='w1_tp2'][state].values[0]
            p2_end = tphdf[tphdf['Phase Loc']=='w1_tpeak'][state].values[0]
            p3_start = tphdf[tphdf['Phase Loc']=='w1_tpeak'][state].values[0]
            p3_end = tphdf[tphdf['Phase Loc']=='w1_tp3'][state].values[0]
            p4_start = tphdf[tphdf['Phase Loc']=='w1_tp3'][state].values[0]
            p4_end = tphdf[tphdf['Phase Loc']=='w1_tp4'][state].values[0]
            p5_start = tphdf[tphdf['Phase Loc']=='w1_tp4'][state].values[0]
            p5_end = tphdf[tphdf['Phase Loc']=='w1_tpend'][state].values[0]
            if p1_end!=0:
                ptdf = tdf[(tdf['week']>=p1_start) & (tdf['week']<=p1_end)]
                p,q = generateCausations(ptdf)
                ndf[k] = enterDict(state, lag, 'Phase1', p,q)
                k = k + 1
            else:
                ptdf = tdf[tdf['week']>=p1_start]
                p,q = generateCausations(ptdf)
                ndf[k] = enterDict(state, lag, 'Phase1', p,q)
                k = k + 1
                continue
            if p2_end!=0:

                ptdf = tdf[(tdf['week']>=p2_start) & (tdf['week']<=p2_end)]
                p,q = generateCausations(ptdf)
                ndf[k] = enterDict(state, lag, 'Phase2',p,q)
                k = k + 1
            else:
                ptdf = tdf[tdf['week']>=p2_start]
                p,q = generateCausations(ptdf)
                ndf[k] = enterDict(state, lag, 'Phase2', p,q)
                k = k + 1
                continue
            if p3_end!=0:
                ptdf = tdf[(tdf['week']>=p3_start) & (tdf['week']<=p3_end)]
                p,q = generateCausations(ptdf)
                ndf[k] = enterDict(state, lag, 'Phase3', p,q)
                k = k + 1
            else:
                ptdf = tdf[tdf['week']>=p3_start]
                p,q = generateCausations(ptdf)
                ndf[k] = enterDict(state, lag, 'Phase3', p,q)
                k = k + 1
                continue
            if p4_end!=0:
                ptdf = tdf[(tdf['week']>=p4_start) & (tdf['week']<=p4_end)]
                p,q = generateCausations(ptdf)
                ndf[k] = enterDict(state, lag, 'Phase4', p,q)
                k = k + 1
            else:
                ptdf = tdf[tdf['week']>=p4_start]
                p,q = generateCausations(ptdf)
                ndf[k] = enterDict(state, lag, 'Phase4', p,q)
                k = k + 1
                continue
            if p5_end!=0:
                ptdf = tdf[(tdf['week']>=p5_start) & (tdf['week']<=p5_end)]
                p,q = generateCausations(ptdf)
                ndf[k] = enterDict(state, lag, 'Phase5', p,q)
                k = k + 1
            else:
                ptdf = tdf[tdf['week']>=p5_start]
                p,q = generateCausations(ptdf)
                ndf[k] = enterDict(state, lag, 'Phase5', p,q)
                k = k + 1
                continue


Alabama
[ 79.         101.14285714  97.85714286  66.85714286  51.14285714
  44.42857143  39.57142857  48.14285714  54.16666667  70.
  81.71428571  77.85714286  84.85714286  74.5         87.
  89.71428571  92.          88.57142857  86.85714286  93.42857143
  92.28571429  84.42857143] [ 1988.  2480.  3072.  3772.  4582.  5494.  6485.  7516.  8526.  9441.
 10178. 10661. 10835. 10678. 10211.  9485.  8577.  7570.  6539.  5544.
  4628.  3812.]

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.7041  , p=0.2082  , df_denom=18, df_num=1
ssr based chi2 test:   chi2=1.9881  , p=0.1585  , df=1
likelihood ratio test: chi2=1.8996  , p=0.1681  , df=1
parameter F test:         F=1.7041  , p=0.2082  , df_denom=18, df_num=1

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=23.7952 , p=0.0028  , df_denom=6, df_num=1
ssr based chi2 test:   chi2=35.6928 , p=0.0000  , df=1
likelihood ratio test: chi2=14.4233 , p=0.0001  , df=1
parameter F test:         F=23.

In [19]:
cdf = pd.DataFrame.from_dict(ndf, 'index')

cdf_F = cdf.pivot_table(index='Phase', columns=['state','lag'], values='Causality_F')
cdf_F.to_csv('Phases_Casality_All_States_FValue_12112020.csv')

cdf_P = cdf.pivot_table(index='Phase', columns=['state','lag'], values='Causality_P')
cdf_P.to_csv('Phases_Casality_All_States_PValue_12112020.csv')

In [21]:
writer = pd.ExcelWriter('Correlation_DesCartes_Incidence_Phases_GrangersCausality_FValue_12112020.xlsx', engine='xlsxwriter')
ncdf = pd.DataFrame.from_dict(ndf, 'index')
phases = ncdf['Phase'].unique()
stateFrame = pd.read_csv('Data/PeakMonths.csv')
for phase in phases:
    twrt = ncdf[ncdf['Phase']==phase]
    twrt_pivot = twrt.pivot_table(index='state', columns='lag', values='Causality_F').reset_index()
    #print(state, phase)
    #print(twrt_pivot.head())
    #print(stateFrame.head())
    twrt_pivot = pd.merge(stateFrame, twrt_pivot, how='left')
    twrt_pivot.sort_values('state', inplace=True)
    twrt_pivot.to_excel(writer, index=False, sheet_name=phase)

writer.save()